In [ ]:
# https://www.analyticsvidhya.com/blog/2018/08/dask-big-datasets-machine_learning-python/

In [ ]:
import dask.dataframe as dd

column_types = {
    'nr.employed': 'float64'
}

%time df = dd.read_csv("../data/bank-additional-full.csv", delimiter=";", dtype=column_types)



In [ ]:
df.select_dtypes(include=['object']).columns.values

In [ ]:
# encode object columns for training
from sklearn.preprocessing import LabelEncoder

columns_to_encode = df.select_dtypes(include=['object']).columns.values

for col in columns_to_encode:
    labelEncodedSeries = dd.from_array(LabelEncoder().fit_transform(df[col]))
    df = df.drop(col, axis=1)
    df = df.merge(labelEncodedSeries.to_frame())
    df = df.rename(columns={0:col})
    
print(df.columns)
df.head()

In [ ]:
import pandas as pd
%time temp = pd.read_csv("../data/bank-additional-full.csv", delimiter=";")

In [ ]:
temp.head()

In [ ]:
df.groupby(df.job).age.mean().compute()

In [ ]:
# train test split
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1].values.compute(),
   df['y'].compute(),
   test_size = 0.4,
   random_state = 123)

In [ ]:
# train parallel with dask
from sklearn.externals.joblib import parallel_backend
from dask.distributed import Client 
from sklearn.ensemble import RandomForestClassifier
client = Client() # start a local Dask client

with parallel_backend('dask'):
    # Your normal scikit-learn code here    
    clf = RandomForestClassifier(max_depth=2, random_state=0)
    %time clf.fit(X_train,y_train)

In [ ]:
# train with sklearn
clf = RandomForestClassifier(max_depth=2, random_state=0, n_jobs=-1)
%time clf.fit(X_train,y_train)